# BigQuery APIs

Google Cloud Datalab provides an integrated environment for working with Google BigQuery for both ad-hoc, exploratory work as well as pipeline development. You've already seen the use of `%%bq` in the [Hello BigQuery](./Hello BigQuery.ipynb) notebook, and various commands in the [BigQuery Commands](./BigQuery Commands.ipynb) notebook.

These BigQuery commands are in fact built using the same BigQuery APIs that are available for your own use. This notebook introduces some examples of using those APIs.

## Importing the API

The Datalab APIs are provided in the `google.datalab` Python module, and the BigQuery functionality is contained within the `google.datalab.bigquery` module.

In [1]:
import google.datalab.bigquery as bq

# Querying Data

The most important BigQuery-related API is the one that allows you to execute a SQL query. The `google.datalab.bigquery.Query` class provides that functionality. To run a query using BigQuery Standard SQL, create a new `Query` object with the desired SQL string, or use an object that has already been defined by the `%%bq query --name` command. Let's take a look at an example:

In [2]:
# Create and run a SQL query
httplogs_query = bq.Query('SELECT * FROM `cloud-datalab-samples.httplogs.logs_20140615` LIMIT 3')

Let's run the query created above with caching turned off, so we're sure to be able to retrieve metadata, such as bytes processed from resulting query job.

For this, we'll need to use a `QueryOutput` object.

In [3]:
output_options = bq.QueryOutput.table(use_cache=False)
result = httplogs_query.execute(output_options=output_options).result()
result

QueryResultsTable job_TSvMKPe0_YW_HH4uAEFCNxnGQ-JU

The result object is a `QueryResultsTable` class, and can be enumerated in the same manner a regular Python list, in addition to retrieving metadata about the result.

In [4]:
# Inspecting the result, and the associated job
print(result.sql)
print(str(result.length) + ' rows')
print(str(result.job.bytes_processed) + ' bytes processed')

SELECT * FROM `cloud-datalab-samples.httplogs.logs_20140615` LIMIT 3
3 rows
24152138 bytes processed


In [5]:
# Inspect the programmatic representation.
# Converting the QueryResultsTable to a vanilla list enables viewing the literal data,
# as well as side-stepping the HTML rendering seen above.
list(result)

[{'timestamp': datetime.datetime(2014, 6, 15, 8, 12, 39, 711942),
  'latency': 256,
  'status': 200,
  'method': 'GET',
  'endpoint': 'Home'},
 {'timestamp': datetime.datetime(2014, 6, 15, 10, 26, 53, 442199),
  'latency': 256,
  'status': 200,
  'method': 'GET',
  'endpoint': 'Home'},
 {'timestamp': datetime.datetime(2014, 6, 15, 13, 4, 58, 103063),
  'latency': 256,
  'status': 200,
  'method': 'GET',
  'endpoint': 'Home'}]

The `QueryResultsTable` has more functionality you can explore, such as converting it to a pandas dataframe, or exporting to a file.

In [6]:
result.to_dataframe()

,timestamp,latency,status,method,endpoint
0,2014-06-15 08:12:39.711942,256,200,GET,Home
1,2014-06-15 10:26:53.442199,256,200,GET,Home
2,2014-06-15 13:04:58.103063,256,200,GET,Home


In [7]:
type(result.to_dataframe())

pandas.core.frame.DataFrame

### Sampling Data

Let's take a look at a sampling query. Consider the following query:

In [8]:
UniqueNames2013 = bq.Query(sql='''
  WITH UniqueNames2013 AS
  (SELECT DISTINCT name
    FROM `bigquery-public-data.usa_names.usa_1910_2013`
    WHERE Year = 2013)
  SELECT * FROM UniqueNames2013
''')

To execute the query and view a sample from the result table, use a `Sampling` object, let's use random sampling for this example:

In [9]:
sampling = bq.Sampling.random(percent=2)
job = UniqueNames2013.execute(sampling=sampling)
job.result()

QueryResultsTable job_6D7JK_FmeC4CeG330YnVJLKfuo1n

Notice every time we run the query above, we get a different set of results, since we chose a random sampling of 2%.

We can also run the query and copy the sampled result into a pandas DataFrame directly. For that, we use a `QueryOutput` object of type `dataframe`:

In [10]:
output_options = bq.QueryOutput.dataframe(max_rows=10)
job = UniqueNames2013.execute(output_options=output_options)

In [11]:
job.result()

,name
0,Lily
1,Aubrey
2,Lillian
3,Ella
4,Skylar
5,Angel
6,Faith
7,Zoe
8,Lola
9,Heidi


# Datasets and Tables

In addition to executing queries, BigQuery objects like Datasets, Tables and their Schemas can be accessed programmatically as well.

## Listing Resources

In [12]:
from google.datalab import Context
datasets = bq.Datasets(Context('cloud-datalab-samples', Context.default().credentials))
for ds in datasets:
  print(ds.name)

DatasetName(project_id='cloud-datalab-samples', dataset_id='appenginelogs')
DatasetName(project_id='cloud-datalab-samples', dataset_id='carprices')
DatasetName(project_id='cloud-datalab-samples', dataset_id='httplogs')


In [13]:
sample_dataset = list(datasets)[1]
tables = sample_dataset.tables()
for table in tables:
  print('%s (%d rows - %d bytes)' % (table.name.table_id, table.metadata.rows, table.metadata.size))

testing (100 rows - 4586 bytes)
training (417 rows - 19086 bytes)


In [14]:
table = bq.Table('cloud-datalab-samples.httplogs.logs_20140615')
fields = map(lambda tsf: tsf.name, table.schema)
list(fields)

['timestamp', 'latency', 'status', 'method', 'endpoint']

## Creating Resources

In [15]:
# Create a new dataset (this will be deleted later in the notebook)
sample_dataset = bq.Dataset('apisample')
sample_dataset.create(friendly_name = 'Sample Dataset', description = 'Created from Sample Notebook')
sample_dataset.exists()

True

In [16]:
# To create a table, we also need to create a schema.
# We've seen before how to create a schema from some existing data,
# let's now try creating it from a list of records:
schema = [
  {'name': 'name', 'type': 'STRING'},
  {'name': 'value', 'type': 'INT64'},
  {'name': 'flag', 'type': 'BOOL', 'mode': 'NULLABLE'}
]
sample_schema = bq.Schema.from_data(schema)

sample_table = bq.Table("apisample.sample_table").create(schema = sample_schema, overwrite = True)

In [17]:
sample_table.schema

BigQuery Schema - Fields:
[{'name': 'name',
  'type': 'STRING'},
 {'name': 'value',
  'type': 'INT64'},
 {'mode': 'NULLABLE',
  'name': 'flag',
  'type': 'BOOL'}]

You can run the cell, below, to see the contents of the new dataset:

In [18]:
list(sample_dataset.tables())

[BigQuery Table - name: ajhamilton-scratch.apisample.sample_table]

## Deleting Resources

In [19]:
# Clear out sample resources
sample_dataset.delete(delete_contents = True)

# Looking Ahead

This notebook covered a small subset of the APIs. Subsequent notebooks cover additional capabilities, such as importing and exporting data into and from BigQuery tables.